In [1]:
import os
import sys
import statistics

aoc_year, aoc_day = os.getcwd().split(os.sep)[-2:]

# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year {aoc_year} --day {aoc_day}

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 14, 2022...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


> Paste part 1 header here

> _Paste part 1 objective here_

In [2]:
from pprint import pprint

with open('input.txt', 'rt') as f:
    lines = [x.strip() for x in f.readlines()]

# Verify parse
pprint(lines[:6])

['494,23 -> 494,17 -> 494,23 -> 496,23 -> 496,20 -> 496,23 -> 498,23 -> 498,16 '
 '-> 498,23 -> 500,23 -> 500,18 -> 500,23 -> 502,23 -> 502,19 -> 502,23 -> '
 '504,23 -> 504,18 -> 504,23',
 '496,36 -> 496,34 -> 496,36 -> 498,36 -> 498,30 -> 498,36 -> 500,36 -> 500,31 '
 '-> 500,36 -> 502,36 -> 502,27 -> 502,36 -> 504,36 -> 504,29 -> 504,36 -> '
 '506,36 -> 506,34 -> 506,36 -> 508,36 -> 508,34 -> 508,36 -> 510,36 -> 510,30 '
 '-> 510,36 -> 512,36 -> 512,30 -> 512,36',
 '511,79 -> 511,77 -> 511,79 -> 513,79 -> 513,70 -> 513,79 -> 515,79 -> 515,73 '
 '-> 515,79 -> 517,79 -> 517,77 -> 517,79 -> 519,79 -> 519,77 -> 519,79 -> '
 '521,79 -> 521,76 -> 521,79',
 '511,79 -> 511,77 -> 511,79 -> 513,79 -> 513,70 -> 513,79 -> 515,79 -> 515,73 '
 '-> 515,79 -> 517,79 -> 517,77 -> 517,79 -> 519,79 -> 519,77 -> 519,79 -> '
 '521,79 -> 521,76 -> 521,79',
 '520,82 -> 524,82',
 '496,164 -> 500,164']


In [3]:
def parse_rockline(line):
    coords = line.split(" -> ")
    coords = [c.split(",") for c in coords]
    for i in range(len(coords)):
        x,y = coords[i]
        coords[i] = [int(x), int(y)]
    return coords

rocklines = [parse_rockline(line) for line in lines]
rocklines
#max([coord[0] for coord in rocklines])

[[[494, 23],
  [494, 17],
  [494, 23],
  [496, 23],
  [496, 20],
  [496, 23],
  [498, 23],
  [498, 16],
  [498, 23],
  [500, 23],
  [500, 18],
  [500, 23],
  [502, 23],
  [502, 19],
  [502, 23],
  [504, 23],
  [504, 18],
  [504, 23]],
 [[496, 36],
  [496, 34],
  [496, 36],
  [498, 36],
  [498, 30],
  [498, 36],
  [500, 36],
  [500, 31],
  [500, 36],
  [502, 36],
  [502, 27],
  [502, 36],
  [504, 36],
  [504, 29],
  [504, 36],
  [506, 36],
  [506, 34],
  [506, 36],
  [508, 36],
  [508, 34],
  [508, 36],
  [510, 36],
  [510, 30],
  [510, 36],
  [512, 36],
  [512, 30],
  [512, 36]],
 [[511, 79],
  [511, 77],
  [511, 79],
  [513, 79],
  [513, 70],
  [513, 79],
  [515, 79],
  [515, 73],
  [515, 79],
  [517, 79],
  [517, 77],
  [517, 79],
  [519, 79],
  [519, 77],
  [519, 79],
  [521, 79],
  [521, 76],
  [521, 79]],
 [[511, 79],
  [511, 77],
  [511, 79],
  [513, 79],
  [513, 70],
  [513, 79],
  [515, 79],
  [515, 73],
  [515, 79],
  [517, 79],
  [517, 77],
  [517, 79],
  [519, 79],
  [519, 7

In [4]:
def generate_grid(rocklines, width = 600, depth=200):
    grid = [["." for col in range(width)] for row in range(depth)]

    for rockline in rocklines:
        #print(rockline)
        for i in range(1, len(rockline)):
            x0,y0 = rockline[i-1]
            x1,y1 = rockline[i]
            #print(rockline[i-1], "->", rockline[i], "vert" if x0 == x1 else "horz")
            if x0 == x1:
                # vertical 
                dy = int((y1-y0)/abs(y1-y0))
                for y in range(y0, y1+dy, dy):
                    grid[y][x0] = "#"
            else:
                # horizontal
                dx = int((x1-x0)/abs(x1-x0))
                for x in range(x0, x1+dx, dx):
                    grid[y0][x] = "#"
    return grid

def dump_grid(grid, i):
    with open(f'grid_{i:05d}.txt', 'wt') as f:
        for line in grid:
            f.write("".join(line) + "\n")
        f.write(str(i))


grid = generate_grid(rocklines)

dump_grid(grid, 0)

In [5]:
units = 0
def find_resting_depth(grid, x = 500, starting_depth=0, verbose=False):
    # find depth under x
    depth_x = [1 if y > starting_depth and grid[y][x] != "." else 0 for y in range(len(grid))].index(1) - 1
    if verbose: print(f"falling at {x},{starting_depth} resting {depth_x}")
    if grid[depth_x+1][x-1] == ".":
        # can fall left
        return find_resting_depth(grid, x-1, depth_x)
    elif grid[depth_x+1][x+1] == ".":
        # can fall right
        return find_resting_depth(grid, x+1, depth_x)
    else:
        # otherwise x
        return x,depth_x

count = 0
while count < 9999:
    try:
        x,y = find_resting_depth(grid)
        count += 1
        grid[y][x] = "o"
        if count % 100 == 0:
            dump_grid(grid, count)
    except:
        print(count)
        break

dump_grid(grid, count)

614


In [6]:
answer1 = count
print("answer 1:", answer1)

answer 1: 614


----

In [7]:
# Download part 2
!aoc download --description-only --overwrite --puzzle-file README.md --year {aoc_year} --day {aoc_day}

Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 14, 2022...
Saving puzzle description to "README.md"...
Done!


\--- Part Two ---
----------


You realize you misread the scan. There isn't an endless void at the bottom of the scan - there's floor, and you're standing on it!

You don't have time to scan the floor, so assume the floor is an infinite horizontal line with a `y` coordinate equal to *two plus the highest `y` coordinate* of any point in your scan.

In the example above, the highest `y` coordinate of any point is `9`, and so the floor is at `y=11`. (This is as if your scan contained one extra rock path like `-infinity,11 -> infinity,11`.) With the added floor, the example above now looks like this:

```
        ...........+........
        ....................
        ....................
        ....................
        .........#...##.....
        .........#...#......
        .......###...#......
        .............#......
        .............#......
        .....#########......
        ....................
<-- etc #################### etc -->

```

To find somewhere safe to stand, you'll need to simulate falling sand until a unit of sand comes to rest at `500,0`, blocking the source entirely and stopping the flow of sand into the cave. In the example above, the situation finally looks like this after `*93*` units of sand come to rest:

```
............o............
...........ooo...........
..........ooooo..........
.........ooooooo.........
........oo#ooo##o........
.......ooo#ooo#ooo.......
......oo###ooo#oooo......
.....oooo.oooo#ooooo.....
....oooooooooo#oooooo....
...ooo#########ooooooo...
..ooooo.......ooooooooo..
#########################

```

Using your scan, simulate the falling sand until the source of the sand becomes blocked. *How many units of sand come to rest?*

In [8]:
deepest_rock = 0
for i, row in enumerate(grid):
    if "#" in row:
        #print(i, row)
        deepest_rock = i

floor = deepest_rock + 2
print(deepest_rock)
MAX_WIDTH = 1000

rocklines.append([[0,floor], [MAX_WIDTH-1, floor]])

grid = generate_grid(rocklines, width = MAX_WIDTH)
dump_grid(grid, 0)

164


In [9]:
count = 0

while count < 100_000:
    x,y = find_resting_depth(grid)
    count += 1
    grid[y][x] = "o"
    if count % 1_000 == 0:
        dump_grid(grid, count)
    if y == 0:
        break
    
dump_grid(grid, count)
print(count)


26170


In [10]:
answer2 = count
print("answer 2:", answer2)

answer 2: 26170
